In [ ]:

# !pip install pypdf langchain_community rapidocr-onnxruntime langchain_openai faiss-cpu langchain

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


#### Assign the Open AI Key

#### Give the path of the PDF Document

In [ ]:
loader = PyPDFLoader("document.pdf", extract_images=True)
pages = loader.load()
# pages[4].page_content

#### Create Retriever

In [ ]:
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

In [ ]:
retriever = faiss_index.as_retriever()

#### Assign the LLM

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

#### Create a History aware retriever chain

In [ ]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

#### Create a pipeline

In [ ]:
### Answer question ###
qa_system_prompt = """
You are an assistant for creating MCQ questions based on the context provided. \
Use the following pieces of retrieved context to formulate the MCQ question. \
Don't repeat the question. \
Display one MCQ question and options only at a time. \
Question in one line and options in separate line and wait for the user Answer. \
If user answer correct say your answer is correct, and ask would you like to try another question from same topic.\
If user answer is wrong say your answer is wrong and display the correct answer with explanation, and ask would you like to try another question from same topic.\
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


rag_chain = create_retrieval_chain(retriever_chain, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

#### Invoke the pipeline

In [ ]:
while True:
    user_input = input("Enter the response: ")
    if user_input == "exit":
        print("bye!")
        break
    print(conversational_rag_chain.invoke( {"input": user_input}, config={"configurable": {"session_id": "abc123"}},)["answer"])

Parent run 83278ad2-ae2d-4e8e-9ce7-2aa47150ed70 not found for run e7f87cba-b086-41c8-aa5f-994ddcddf98f. Treating as a root run.


What is the focus of the attention mechanism in the encoder self-attention at layer 5 of 6 in the provided context?
A) Verb 'making'
B) Noun 'laws'
C) Adjective 'difficult'
D) Adverb 'more'
E) Preposition 'of'

Answer the question by selecting one of the options (A, B, C, D, E).


Parent run 8e0d644e-8e29-47f9-a2cd-5f2fc7629ba1 not found for run 0e4f4253-f223-4059-b38e-1127267f68a2. Treating as a root run.


Your answer is correct!

Would you like to try another question from the same topic?


Parent run 0c587ed8-4fec-41fb-8792-50f71497c3e8 not found for run 770577c7-8441-4941-925d-388c7d6cec41. Treating as a root run.


...

What is the key advantage of the Transformer model architecture over models using recurrent or convolutional layers for translation tasks?
A) Faster training
B) Higher memory utilization
C) Lower computational complexity
D) Improved interpretability
E) Enhanced regularization

Answer the question by selecting one of the options (A, B, C, D, E).


Parent run 83743fbd-b0bf-4ca2-9a8d-665213c19c50 not found for run 22b71861-eccc-47d5-bb2f-d465ff78649e. Treating as a root run.


Your answer is incorrect.

The correct answer is:
A) Faster training

The key advantage of the Transformer model architecture over models using recurrent or convolutional layers for translation tasks is that it can be trained significantly faster.

Would you like to try another question from the same topic?


Parent run d00e56a9-c955-40b6-afe2-118e1cd2f47a not found for run 8fe1681b-fda3-4da6-858d-6da0aacfafd2. Treating as a root run.


What is the primary focus of the attention mechanism in the encoder self-attention at layer 5 of 6 in the provided context?
A) Verb 'making'
B) Noun 'laws'
C) Adjective 'difficult'
D) Adverb 'more'
E) Preposition 'of'

Answer the question by selecting one of the options (A, B, C, D, E).
bye!
